In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')

train = train.loc[:,['Survived','Pclass','Sex','Age','Fare']]
train.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [2]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
dtype: int64

In [3]:
train = train.dropna(subset=['Age']).reset_index(drop=True)
len(train)

714

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train['Sex'] = encoder.fit_transform(train['Sex'].values)
train.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500


In [5]:
from sklearn.preprocessing import StandardScaler

standard = StandardScaler()
train_std = pd.DataFrame(standard.fit_transform(train.loc[:,['Age','Fare']]),columns=['Age','Fare'])

train['Age'] = train_std['Age']
train['Fare'] = train_std['Fare']

train.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,1,-0.530377,-0.518978
1,1,1,0,0.571831,0.691897
2,1,3,0,-0.254825,-0.506214
3,1,1,0,0.365167,0.348049
4,0,3,1,0.365167,-0.503850


In [6]:
from sklearn.svm import LinearSVC

x_train = train.loc[:,['Pclass','Sex','Age','Fare']].values
y_train = train.loc[:,['Survived']].values

y_train = np.reshape(y_train,(-1))

model = LinearSVC(random_state=1)

In [7]:
model.fit(x_train,y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(random_state=1)

In [8]:
test = pd.read_csv('test.csv')

test_index = test.loc[:,['PassengerId']]
test = test.loc[:,['Pclass','Sex','Age','Fare']]

test['Sex'] = encoder.transform(test['Sex'].values)

test_std = pd.DataFrame(standard.transform(test.loc[:,['Age','Fare']]),columns=['Age','Fare'])

test['Age'] = test_std['Age']
test['Fare'] = test_std['Fare']

test = test.fillna({'Age':0,'Fare':0})

test.head()

,Pclass,Sex,Age,Fare
0,3,1,0.330723,-0.508025
1,3,0,1.191823,-0.523705
2,2,1,2.225142,-0.472885
3,3,1,-0.185937,-0.492267
4,3,0,-0.530377,-0.423718


In [9]:
x_test = test.values
y_test = model.predict(x_test)

In [10]:
output = pd.concat([test_index,pd.DataFrame(y_test,columns=['Survived'])],axis=1)

output.to_csv('result.csv',index=False)